# 2. Реализация Feature Store

1. Определите фичи для offline и online:
- Какие фичи будут использоваться в обучении?
- Как они будут рассчитываться в реальном времени?
- Какие фичи критичны для согласованности?

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [4]:
df = pd.read_csv("../data/hospital_readmissions_30k.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   patient_id             30000 non-null  int64  
 1   age                    30000 non-null  int64  
 2   gender                 30000 non-null  object 
 3   blood_pressure         30000 non-null  object 
 4   cholesterol            30000 non-null  int64  
 5   bmi                    30000 non-null  float64
 6   diabetes               30000 non-null  object 
 7   hypertension           30000 non-null  object 
 8   medication_count       30000 non-null  int64  
 9   length_of_stay         30000 non-null  int64  
 10  discharge_destination  30000 non-null  object 
 11  readmitted_30_days     30000 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.7+ MB
None


,patient_id,age,gender,blood_pressure,cholesterol,bmi,diabetes,hypertension,medication_count,length_of_stay,discharge_destination,readmitted_30_days
0,1,74,Other,130/72,240,31.5,Yes,No,5,1,Nursing_Facility,Yes
1,2,46,Female,120/92,292,36.3,No,No,4,3,Nursing_Facility,No
2,3,89,Other,135/78,153,30.3,No,Yes,1,1,Home,No
3,4,84,Female,123/80,153,31.5,No,Yes,3,10,Home,No
4,5,32,Other,135/84,205,18.4,No,Yes,6,4,Nursing_Facility,No


In [6]:
basic_features = ['age', 'gender']
medical_features = ['blood_pressure', 'cholesterol', 'bmi', 'diabetes', 'hypertension']
treatment_features = ['medication_count', 'length_of_stay', 'discharge_destination']
target_feature = ['readmitted_30_days']

print("Базовые фичи:")
print(f"  - {basic_features}")

print("\nМедицинские фичи:")
print(f"  - {medical_features}")

print("\nФичи лечения:")
print(f"  - {treatment_features}")

print("\nЦелевая переменная:")
print(f"  - {target_feature}")

Базовые фичи:
  - ['age', 'gender']

Медицинские фичи:
  - ['blood_pressure', 'cholesterol', 'bmi', 'diabetes', 'hypertension']

Фичи лечения:
  - ['medication_count', 'length_of_stay', 'discharge_destination']

Целевая переменная:
  - ['readmitted_30_days']


In [7]:
offline_features = [
    'age', 'gender', 'blood_pressure', 'cholesterol', 'bmi', 
    'diabetes', 'hypertension', 'medication_count', 'length_of_stay',
    'discharge_destination', 'readmitted_30_days'
]

online_features = [
    'age', 'gender', 'blood_pressure', 'cholesterol', 'bmi',
    'diabetes', 'hypertension', 'medication_count', 'discharge_destination'
]

print("OFFLINE фичи (для обучения):")
for feat in offline_features:
    print(f"  ✓ {feat}")

print("\nONLINE фичи (для реального времени):")
for feat in online_features:
    print(f"  ✓ {feat}")

print(f"\nКритические для согласованности: {online_features}")

OFFLINE фичи (для обучения):
  ✓ age
  ✓ gender
  ✓ blood_pressure
  ✓ cholesterol
  ✓ bmi
  ✓ diabetes
  ✓ hypertension
  ✓ medication_count
  ✓ length_of_stay
  ✓ discharge_destination
  ✓ readmitted_30_days

ONLINE фичи (для реального времени):
  ✓ age
  ✓ gender
  ✓ blood_pressure
  ✓ cholesterol
  ✓ bmi
  ✓ diabetes
  ✓ hypertension
  ✓ medication_count
  ✓ discharge_destination

Критические для согласованности: ['age', 'gender', 'blood_pressure', 'cholesterol', 'bmi', 'diabetes', 'hypertension', 'medication_count', 'discharge_destination']


2. Реализуйте Feature Store с помощью Feast:

In [2]:
df_features = pd.read_parquet('./hospital_readmissions/feature_repo/data/patient_features.parquet')
df_features.head()

,patient_id,age,cholesterol,bmi,medication_count,length_of_stay,systolic_bp,diastolic_bp,gender,diabetes,...,multiple_conditions,age_medication_interaction,bmi_cholesterol_interaction,cardiovascular_risk,treatment_intensity,high_cholesterol,high_bmi,extended_stay,event_timestamp,created_timestamp
0,1,74,240,31.5,5,1,130,72,Other,Yes,...,1,370,7560.0,5.650,5,1,1,0,2025-11-19 14:50:16.864719,2025-11-19 14:50:16.864719
1,2,46,292,36.3,4,3,120,92,Female,No,...,0,184,10599.6,6.290,12,1,1,0,2025-11-19 14:50:16.864719,2025-11-19 14:50:16.864719
2,3,89,153,30.3,1,1,135,78,Other,No,...,1,89,4635.9,5.145,1,0,1,0,2025-11-19 14:50:16.864719,2025-11-19 14:50:16.864719
3,4,84,153,31.5,3,10,123,80,Female,No,...,1,252,4819.5,5.145,30,0,1,1,2025-11-19 14:50:16.864719,2025-11-19 14:50:16.864719
4,5,32,205,18.4,6,4,135,84,Other,No,...,1,192,3772.0,4.215,24,1,0,0,2025-11-19 14:50:16.864719,2025-11-19 14:50:16.864719


3. Определите фичи в features.py

[features.py](hospital_readmissions\feature_repo\features.py)

cd hospital_readmissions/feature_repo

feast apply

Created project hospital_readmissions

Created entity patient

Created feature view demographic_features

Created feature view interaction_features

Created feature view patient_stats

WARNING:root:Cannot use sqlite_vec for vector search

WARNING:root:Cannot use sqlite_vec for vector search

WARNING:root:Cannot use sqlite_vec for vector search

Created sqlite table hospital_readmissions_demographic_features

Created sqlite table hospital_readmissions_interaction_features

Created sqlite table hospital_readmissions_patient_stats

feast materialize --disable-event-timestamp

4. Реализуйте согласованность фич:
5. Проверьте согласованность фич:

In [ ]:
from feast import FeatureStore

print("Проверка Feature Store")
store = FeatureStore("./feature_repo")

# Смотрим, что зарегистрировано
print("entities:")
for entity in store.list_entities():
    print(f"  - {entity.name}")

print("\nfeature views:")
for fv in store.list_feature_views():
    print(f"  - {fv.name}")
    for feature in fv.features:
        print(f"    * {feature.name} ({feature.dtype})")

Проверка Feature Store
entities:
  - patient

feature views:
  - demographic_features
    * gender (String)
    * diabetes (String)
    * hypertension (String)
    * discharge_destination (String)
    * bmi_category (String)
    * age_group (String)
  - interaction_features
    * high_bp_high_chol (Int32)
    * multiple_conditions (Int32)
    * age_medication_interaction (Float32)
    * bmi_cholesterol_interaction (Float32)
  - patient_stats
    * age (Int32)
    * cholesterol (Int32)
    * bmi (Float32)
    * medication_count (Int32)
    * length_of_stay (Int32)
    * systolic_bp (Int32)
    * diastolic_bp (Int32)
    * bp_ratio (Float32)
    * cardiovascular_risk (Float32)
    * treatment_intensity (Float32)
    * high_cholesterol (Int32)
    * high_bmi (Int32)
    * extended_stay (Int32)


In [ ]:
# Тест получения offline фич (для обучения)
import pandas as pd

entity_df = pd.DataFrame({
    "patient_id": [1, 2, 3, 4, 5]
})

try:
    training_df = store.get_historical_features(
        entity_df=entity_df,
        features=[
            "patient_stats:age",
            "patient_stats:cholesterol", 
            "patient_stats:bmi",
            "patient_stats:medication_count",
            "demographic_features:gender",
            "demographic_features:diabetes",
            "demographic_features:hypertension"
        ]
    ).to_df()
    
    print("✓ Offline фичи получены успешно")
    print("Пример данных:")
    print(training_df.head())
    
except Exception as e:
    print(f"✗ Ошибка при получении offline фич: {e}")

✓ Offline фичи получены успешно
Пример данных:
   patient_id                  event_timestamp  age  cholesterol   bmi  \
0           1 2025-11-19 15:23:42.970457+00:00   74          240  31.5   
1           2 2025-11-19 15:23:42.970457+00:00   46          292  36.3   
2           3 2025-11-19 15:23:42.970457+00:00   89          153  30.3   
3           4 2025-11-19 15:23:42.970457+00:00   84          153  31.5   
4           5 2025-11-19 15:23:42.970457+00:00   32          205  18.4   

   medication_count  gender diabetes hypertension  
0                 5   Other      Yes           No  
1                 4  Female       No           No  
2                 1   Other       No          Yes  
3                 3  Female       No          Yes  
4                 6   Other       No          Yes  


In [15]:
# Тест получения online фич (для инференса)

try:
    online_features = store.get_online_features(
        features=[
            "patient_stats:age",
            "patient_stats:cholesterol",
            "patient_stats:bmi", 
            "patient_stats:medication_count",
            "demographic_features:gender",
            "demographic_features:diabetes", 
            "demographic_features:hypertension"
        ],
        entity_rows=[
            {"patient_id": 1},
            {"patient_id": 2},
            {"patient_id": 3}
        ]
    ).to_df()
    
    print("✓ Online фичи получены успешно")
    print("Пример данных:")
    print(online_features.head())
    
except Exception as e:
    print(f"✗ Ошибка при получении online фич: {e}")

✓ Online фичи получены успешно
Пример данных:
   patient_id  age  cholesterol        bmi  medication_count hypertension  \
0           1   74          240  31.500000                 5           No   
1           2   46          292  36.299999                 4           No   
2           3   89          153  30.299999                 1          Yes   

   gender diabetes  
0   Other      Yes  
1  Female       No  
2   Other       No  


In [18]:
# Проверка согласованности между offline и online фичами

# Сравниваем значения для patient_id 1, 2, 3
test_patients = [1, 2, 3]

for patient_id in test_patients:
    offline_vals = training_df[training_df['patient_id'] == patient_id].iloc[0]
    online_vals = online_features[online_features['patient_id'] == patient_id].iloc[0]
    
    print(f"\nПациент {patient_id}:")
    for feature in ['age', 'cholesterol', 'gender', 'medication_count']:
        offline_val = offline_vals[feature]
        online_val = online_vals[feature]
        matches = offline_val == online_val
        status = "✓ СОВПАДАЕТ" if matches else "✗ РАЗЛИЧАЕТСЯ"
        print(f"  {feature}: {offline_val} vs {online_val} - {status}")

print("\n✓ Проверка согласованности завершена")


Пациент 1:
  age: 74 vs 74 - ✓ СОВПАДАЕТ
  cholesterol: 240 vs 240 - ✓ СОВПАДАЕТ
  gender: Other vs Other - ✓ СОВПАДАЕТ
  medication_count: 5 vs 5 - ✓ СОВПАДАЕТ

Пациент 2:
  age: 46 vs 46 - ✓ СОВПАДАЕТ
  cholesterol: 292 vs 292 - ✓ СОВПАДАЕТ
  gender: Female vs Female - ✓ СОВПАДАЕТ
  medication_count: 4 vs 4 - ✓ СОВПАДАЕТ

Пациент 3:
  age: 89 vs 89 - ✓ СОВПАДАЕТ
  cholesterol: 153 vs 153 - ✓ СОВПАДАЕТ
  gender: Other vs Other - ✓ СОВПАДАЕТ
  medication_count: 1 vs 1 - ✓ СОВПАДАЕТ

✓ Проверка согласованности завершена
